# Let's apply the GP-based optimizer to our small Hubbard model.

Make sure your jupyter path is the same as your virtual environment that you used to install all your packages. 
If nopt, do something like this in your terminal:

`$ ipython kernel install --user --name TUTORIAL --display-name "Python 3.9"`

In [ ]:
# check your python
from platform import python_version

print(python_version())

Gaussian Process (GP) models were introduced in the __[Gaussian Process Models](optimization.ipynb)__ notebook. The GP-based optimizer uses these techniques as implemented in the included __[opti_by_gp.py](opti_by_gp.py)__ module, which also provides helpers for plotting results. Note that this module uses the ImFil optimizer underneath, a choice that can not currently be changed.

As a first step, create once more a __[Hubbard Model](hubbard_model_intro.ipynb)__ setup.

In [ ]:
import hubbard as hb
import logging
import noise_model as noise
import numpy as np
import opti_by_gp as obg
from IPython.display import Image

logging.getLogger('hubbard').setLevel(logging.INFO)

In [ ]:
# Select a model appropriate for the machine used:
#    laptop -> use small model
#    server -> use medium model

MODEL = hb.small_model
#MODEL = hb.medium_model

# Hubbard model for fermions (Fermi-Hubbard) required parameters
xdim, ydim, t, U, chem, magf, periodic, spinless = MODEL()

# Number of electrons to add to the system
n_electrons_up   = 1
n_electrons_down = 1
n_electrons = n_electrons_up + n_electrons_down

# Total number of "sites", with each qubit representing occupied or not
spinfactor = spinless and 1 or 2
n_qubits = n_sites = xdim * ydim * spinfactor

# Create the Hubbard Model for use with Qiskit
hubbard_op = hb.hamiltonian_qiskit(
    x_dimension        = xdim,
    y_dimension        = ydim,
    tunneling          = t,
    coulomb            = U,
    chemical_potential = chem,
    magnetic_field     = magf,
    periodic           = periodic,
    spinless           = spinless)

The GP modeling needs persistent access to the evaluated points, so tell the objective to save them. Otherwise, the objective is the same as before. Choose the maximum number of objective evaluations, the initial and set the bounds. Then run the optimization using GP (as mentioned before, this uses ImFil underneath).

In [ ]:
# noise-free objective with enough Trotter steps to get an accurate result
objective = hb.EnergyObjective(hubbard_op, n_electrons_up, n_electrons_down,
    trotter_steps=3, save_evals=True)

# initial and bounds (set good=True to get tighter bounds)
initial_amplitudes, bounds = MODEL.initial(
    n_electrons_up, n_electrons_down, objective.npar(), good=False)

# max number of allowed function evals
maxevals = 100

In [ ]:
result = obg.opti_by_gp(objective.npar(), bounds, objective, maxevals)

In [ ]:
print('Results with GP:')
print("Estimated energy: %.5f" % result[1])
print("Parameters:      ", result[0])
print("Number of iters: ", result[2])

Now let's analyze the results be looking at the sample evaluations and convergence plot.

In [ ]:
Image(filename='samples.png')

The left plot shows:
1) the points sampled with GP (pink squares): you can see that we have some points everywhere in the space, but a denser pink square cloud where the function has its minimum

2) yellow circles (5) -- these are the points from which the local search with ImFil starts: we choose the best point found by the GP, and another 4 points based on their function value and distance to already selected start points. 5 is a parameter, if you want to do only one local search, you can just start from the best point found by the GP iterations. Also: not all 5 points will necessarily be used for ImFil, the optimization stops when the maximum number of allowed evaluations has been reached. 

3) the green squares are the points ImFil decided to sample -- you can see that they cover most of the space. Wouldn't it be nice to force ImFil to search only a smaller radius?!

4) the red dot indicates the best point found during optimization

5) the contours are created by using a GP model and all sample information that we collected - so this is not the true contours, but the best guess of what the true contours may look like

The right plot shows the GP approximation of the energy surface - again, not the true surface, just our best guess based on training a GP on all input-output pairs

In [ ]:
Image(filename='progress.png')

This plot shows the progress we are making with respect to improving the energy versus the number of function evaluations. 
We show the best energy value found so far, thus, the graph is monotonically decreasing and has a step-like shape. whenever the graph is flat, it means that during these iterations no energy improvements were found. If you were to plot simply the energy at each function evaluation, the graph would go up and down because we use sampling based algorithms and not gradient-based algorithms. Thus, not in every iteration we find an improvement. 
There is a large down-step in the beginning - this is due to our random space filling sampling initially. We can also see that ImFil does not make much progress here. The GP-based sampling is used until 30 evaluations. 

Note that the GP based optimizer has parameters, including the size of the initial experimental design, the number of iterations that we want to apply the GP (here 30), the maximum number of local searches with ImFil after the GP is done, .... see the __[opti_by_gp.py](opti_by_gp.py)__ module (or run the cell below to load).

In [ ]:
%load 'opti_by_gp.py'

**Exercise:** redo the above analysis using a noisy objective. If time is limited, consider only using sampling noise, e.g. by setting `shots=8192` (see the notebook on __[noise](hubbard_vqe_noise.ipynb)__ for more examples), and using tight bounds.

**Optional Exercise:** for comparison purposes, follow-up with an optimization run that does not use GP and try in particular what happens when using only few function evaluations (20, say, if using tight bounds). Try different optimizers (but consider that some, such as SPSA, will take more evalations per iteration; and consider that optimizers that do not respect bounds are at a severe disadvantage).

In [ ]:
# noisy objective, adjust as desired
objective = hb.EnergyObjective(hubbard_op, n_electrons_up, n_electrons_down,
    trotter_steps=3, shots=8192, save_evals=True)

# initial and bounds (set good=False to get loose bounds)
initial_amplitudes, bounds = MODEL.initial(
    n_electrons_up, n_electrons_down, objective.npar(), good=True)

# max number of allowed function evals
maxevals = 20

In [ ]:
result = obg.opti_by_gp(objective.npar(), bounds, objective, maxevals)

In [ ]:
print('Results with GP:')
print("Estimated energy: %.5f" % result[1])
print("Parameters:      ", result[0])
print("Number of iters: ", result[2])

In [ ]:
Image(filename='samples.png')

In [ ]:
Image(filename='progress.png')

In [ ]:
# Pull in a couple of optimizers to play with
from qiskit.algorithms.optimizers import COBYLA, SPSA
try:
    from qiskit.algorithms.optimizers import IMFIL, SNOBFIT
except ImportError:
    print("install scikit-quant to use IMFIL and SNOBFIT")

In [ ]:
result = IMFIL(maxiter=maxevals).optimize(
        num_vars = objective.npar(),
        objective_function=objective,
        initial_point = initial_amplitudes,
        variable_bounds = bounds
      )

In [ ]:
print('ImFil results without GP:')
print("Estimated energy: %.5f" % result[1])
print("Parameters:      ", result[0])
print("Number of iters: ", result[2])

In [ ]:
result = SPSA(maxiter=maxevals//2).optimize(    # note division by 2, but also check actual evals!
        num_vars = objective.npar(),
        objective_function=objective,
        initial_point = initial_amplitudes,
        variable_bounds = bounds
      )

In [ ]:
print('SPSA results without GP:')
print("Estimated energy: %.5f" % result[1])
print("Parameters:      ", result[0])
print("Number of iters: ", result[2])